## 4.9 Tasks, Step 1

### Step 3: Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\steve\Documents\CF - Instacart Project'

In [3]:
opm = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_nearfinal.pkl'))

In [4]:
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### Step 4: Wrangle the customer dataset

In [7]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [15]:
# Renaming columns to a new df
custa = cust.rename(columns={'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'num_dependants', 'fam_status':'marital_status'}, inplace = False)

In [19]:
custa.head()

,user_id,gender,state,age,num_dependants,marital_status,income
0,26711,Female,Missouri,48,3,married,165665
1,33890,Female,New Mexico,36,0,single,59285
2,65803,Male,Idaho,35,2,married,99568
3,125935,Female,Iowa,40,0,single,42049
4,130797,Female,Maryland,26,1,married,40374


In [17]:
# Dropping some columns
columns_to_drop = ['date_joined', 'first_name', 'last_name', 'date_joined']

In [18]:
custa.drop(columns=columns_to_drop, inplace=True)

### Step 5: Complete fundamental checks

In [16]:
# Checking consistency and numbers
custa.describe()

,user_id,age,income
count,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,94632.852548
std,59527.555167,18.480962,42473.786988
min,1.000000,18.000000,25903.000000
25%,51553.000000,33.000000,59874.000000
50%,103105.000000,49.000000,93547.000000
75%,154657.000000,66.000000,124244.000000
max,206209.000000,81.000000,593901.000000


In [24]:
# That high income seemed weird, checking it out
custa_sorted = custa.sort_values(by='income', ascending=False)

In [26]:
# I guess its fine, a lot of em are from dual income I guess?
print(custa_sorted)

        user_id first_name   last_name  gender          state  age  \
21726     58596     Thomas       Cohen    Male     Washington   75   
202655    98580        NaN   Patterson    Male     Washington   43   
81583    103702    Kathryn  Mclaughlin  Female      Louisiana   55   
111950   187680  Catherine      Church  Female        Wyoming   43   
167037   189861     Pamela      Jordan  Female   Pennsylvania   47   
...         ...        ...         ...     ...            ...  ...   
67441    151083  Katherine        Rush  Female        Montana   32   
169181   183978    Frances       Nixon  Female        Montana   39   
194354   162338        NaN        Page  Female     California   21   
105031   193849   Patricia        Hart  Female  West Virginia   34   
34047    134975     Robert     Wheeler    Male   North Dakota   39   

                          marital_status  income  
21726                            married  593901  
202655                           married  592409  
81583 

In [27]:
# Checking frequencies for odd data
custa['gender'].value_counts()

gender
Male      104067
Female    102142
Name: count, dtype: int64

In [28]:
custa['state'].value_counts()

state
Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana 

Everything looks good, its VERY odd to me that the entries were so uniform/close in count but maybe that was planned for this dataset? Otherwise, I can't think of a reason why that would be, in terms of recording error.

### Step 6: Combine data

In [20]:
# Getting some numbers to compare after merge
custa.head(1)

,user_id,gender,state,age,num_dependants,marital_status,income
0,26711,Female,Missouri,48,3,married,165665


In [21]:
opm.head(1)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.37,Low spender,20.5,Non-frequent customer


In [32]:
opm.shape

(32404859, 22)

In [33]:
custa.shape

(206209, 8)

In [22]:
# Merge attempt to a new df
df_merged = opm.merge(custa, on = 'user_id', indicator = True)

In [24]:
df_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_days,frequency_flag,gender,state,age,num_dependants,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,3,married,40423,both


In [25]:
df_merged['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

Numbers work out, nothing looks off, going to remove the _merge column.

In [26]:
df_merged.drop(columns=['_merge'], inplace=True)

In [27]:
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'nearfinal.pkl'))

In [28]:
df_merged.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'nearfinal.csv'))